In [ ]:
!pip install geopy yarl

## Extract Elements from a URL

In [3]:
from yarl import URL

<IPython.core.display.Javascript object>

In [4]:
url = URL("https://github.com/search?q=data+science")

<IPython.core.display.Javascript object>

In [5]:
url.scheme

'https'

<IPython.core.display.Javascript object>

In [6]:
url.host

'github.com'

<IPython.core.display.Javascript object>

In [7]:
url.path

'/search'

<IPython.core.display.Javascript object>

In [8]:
url.query_string

'q=data science'

<IPython.core.display.Javascript object>

In [9]:
url = URL("https://github.com/khuyentran1401/Data-science#contents")
url.fragment

'contents'

<IPython.core.display.Javascript object>

In [10]:
import pandas as pd

data = pd.read_csv("URL Classification.csv", names=["url", "Type"], index_col=0)
data

,url,Type
1,http://www.liquidgeneration.com/,Adult
2,http://www.onlineanime.org/,Adult
3,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...,Adult
4,http://www.galeon.com/kmh/,Adult
5,http://www.fanworkrecs.com/,Adult
...,...,...
1562974,http://www.maxpreps.com/,Sports
1562975,http://www.myscore.com/,Sports
1562976,http://sportsillustrated.cnn.com/highschool,Sports
1562977,http://rss.cnn.com/rss/si_highschool?format=xml,Sports


<IPython.core.display.Javascript object>

In [11]:
sample = data.sample(10000, random_state=1)
sample.head(10)

,url,Type
1308349,http://yorkrite.com/ne/gcram/,Society
276825,http://www.alliedartistsofamerica.org/,Arts
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society
392387,http://www.sunnytec.com.tw/,Business
1267730,http://www.kabissa.org,Society
1392788,http://www.newadvent.org/cathen/12134b.htm,Society
672881,http://www.gamespot.com/ps2/driving/arcticthun...,Games
1526125,http://www.studsquad.net/,Sports
1009901,http://www.spacedog.biz,Reference
330964,http://www.meditrans-japan.com/,Business


<IPython.core.display.Javascript object>

In [12]:
sample["url"] = sample["url"].apply(lambda url: URL(url))

<IPython.core.display.Javascript object>

In [17]:
processed = sample.assign(
    host=sample.url.apply(lambda url: url.host),
    path=sample.url.apply(lambda url: url.path),
    name=sample.url.apply(lambda url: url.name),
)
processed

,url,Type,host,path,name
1308349,http://yorkrite.com/ne/gcram/,Society,yorkrite.com,/ne/gcram/,
276825,http://www.alliedartistsofamerica.org/,Arts,www.alliedartistsofamerica.org,/,
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society,www.msstate.edu,/org/farmhouse/index.html,index.html
392387,http://www.sunnytec.com.tw/,Business,www.sunnytec.com.tw,/,
1267730,http://www.kabissa.org,Society,www.kabissa.org,/,
...,...,...,...,...,...
169463,http://www.naxos.com/composerinfo/3303.htm,Arts,www.naxos.com,/composerinfo/3303.htm,3303.htm
1094461,http://members.tripod.com/arroweb1/,Science,members.tripod.com,/arroweb1/,
526843,http://www.petroskills.com/,Business,www.petroskills.com,/,
171473,http://www.gregbartholomew.com,Arts,www.gregbartholomew.com,/,


<IPython.core.display.Javascript object>

In [210]:
group = processed.groupby(["Type", "host"]).agg(count=("url", "count"))
group

count
Type   host                               
Adult  101fetishes.com                   1
       alpha-cow.com                     1
       amateursmut.com                   1
       bombes-orientales.netpass.tv      1
       buzzerotica.tripod.com            1
...                                    ...
Sports www.wsstats.com                   1
       www.wtdefence.com                 1
       www.yantu.com                     1
       www.yarmrugbyclub.co.uk           1
       www2.wilmington.edu               1

[8746 rows x 1 columns]

<IPython.core.display.Javascript object>

In [213]:
sorted_group = group.sort_values(by="count", ascending=False).reset_index()
sorted_group

,Type,host,count
0,Arts,www.geocities.com,113
1,Society,www.newadvent.org,91
2,Arts,www.angelfire.com,69
3,Society,www.geocities.com,56
4,Recreation,www.geocities.com,47
...,...,...,...
8741,Business,www.wilmslow-audio.co.uk,1
8742,Business,www.whitingcorp.com,1
8743,Business,www.whitesuffolk.org.au,1
8744,Business,www.whitebredshorthorn.com,1


<IPython.core.display.Javascript object>

In [218]:
largest = sorted_group.groupby("Type").head(5)
largest

,Type,host,count
0,Arts,www.geocities.com,113
1,Society,www.newadvent.org,91
2,Arts,www.angelfire.com,69
3,Society,www.geocities.com,56
4,Recreation,www.geocities.com,47
...,...,...,...
1339,News,www.scottolsonphotography.com,1
1340,News,www.postindependent.com,1
1341,News,www.pgpost.com,1
1342,News,www.perspectives.ubc.ca,1


<IPython.core.display.Javascript object>

In [255]:
edges = largest[["Type", "host"]]

<IPython.core.display.Javascript object>

In [265]:
def create_node_df(df: pd.DataFrame, col_name: str):
    nodes = (
        df[[col_name]]
        .assign(type=col_name)
        .rename(columns={col_name: "node"})
        .drop_duplicates()
    )
    return nodes

<IPython.core.display.Javascript object>

In [271]:
type_nodes = create_node_df(largest, "Type")
url_nodes = create_node_df(largest, "host")
nodes = pd.concat([type_nodes, url_nodes])
nodes

,node,type
0,Arts,Type
1,Society,Type
4,Recreation,Type
6,Games,Type
9,Computers,Type
11,Sports,Type
13,Science,Type
14,Health,Type
18,Home,Type
28,Kids,Type


<IPython.core.display.Javascript object>

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
PASSWORD = os.getenv("GRAPHISTRY_PASSWORD")
USERNAME = os.getenv("GRAPHISTRY_USERNAME")

In [171]:
import graphistry

graphistry.register(api=3, username=USERNAME, password=PASSWORD)

<IPython.core.display.Javascript object>

In [243]:
types = list(edges.Type.unique())
types

['Arts',
 'Society',
 'Recreation',
 'Games',
 'Computers',
 'Sports',
 'Science',
 'Health',
 'Home',
 'Kids',
 'Shopping',
 'Reference',
 'Business',
 'Adult',
 'News']

<IPython.core.display.Javascript object>

In [279]:
from faker import Faker

fake = Faker()
colors = [fake.color() for _ in range(len(types))]
node_color_mapping = dict(zip(types, colors))
node_color_mapping

{'Arts': '#e56110',
 'Society': '#7a06d8',
 'Recreation': '#f7e1be',
 'Games': '#e07db7',
 'Computers': '#a8dbf4',
 'Sports': '#fffdb5',
 'Science': '#e2917f',
 'Health': '#969e0e',
 'Home': '#ffe3c6',
 'Kids': '#012e77',
 'Shopping': '#96f2aa',
 'Reference': '#f4bc66',
 'Business': '#728bc9',
 'Adult': '#321875',
 'News': '#ffadf8'}

<IPython.core.display.Javascript object>

In [282]:
g = (
    graphistry.edges(edges, "Type", "host")
    .nodes(nodes, "node")
    .encode_point_color(
        "node", categorical_mapping=node_color_mapping, default_mapping="silver"
    )
    .encode_point_icon(
        "type", categorical_mapping={"host": "link", "Type": "newspaper-o"}
    )
)

<IPython.core.display.Javascript object>

In [283]:
g.plot()

<IPython.core.display.Javascript object>